## The Machine Learning Workflow

Machine learning requires experimenting with a wide range of datasets, data preparation steps, and algorithms to build a model that maximizes some target metric. Once you have built a model, you also need to deploy it to a production system, monitor its performance, and continuously retrain it on new data and compare with alternative models.

Being productive with machine learning can therefore be challenging for several reasons:

- **It’s difficult to keep track of experiments.** When you are just working with files on your laptop, or with an interactive notebook, how do you tell which data, code and parameters went into getting a particular result?
- **It’s difficult to reproduce code.** Even if you have meticulously tracked the code versions and parameters, you need to capture the whole environment (for example, library dependencies) to get the same result again. This is especially challenging if you want another data scientist to use your code, or if you want to run the same code at scale on another platform (for example, in the cloud).
- **There’s no standard way to package and deploy models.** Every data science team comes up with its own approach for each ML library that it uses, and the link between a model and the code and parameters that produced it is often lost.

Moreover, although individual ML libraries provide solutions to some of these problems (for example, model serving), to get the best result you usually want to try multiple ML libraries. MLflow lets you train, reuse, and deploy models with any library and package them into reproducible steps that other data scientists can use as a “black box,” without even having to know which library you are using.

## MLflow Components
MLflow provides three components to help manage the ML workflow:

- **MLflow Tracking** is an API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results. You can use MLflow Tracking in any environment (for example, a standalone script or a notebook) to log results to local files or to a server, then compare multiple runs. Teams can also use it to compare results from different users.

- **MLflow Projects** are a standard format for **packaging reusable data science code**. Each project is simply a directory with code or a Git repository, and uses a descriptor file or simply convention to specify its dependencies and how to run the code. For example, projects can contain a conda.yaml file for specifying a Python Conda environment. When you use the MLflow Tracking API in a Project, MLflow automatically remembers the project version executed (for example, Git commit) and any parameters. You can easily run existing MLflow Projects from GitHub or your own Git repository, and chain them into multi-step workflows.

- **MLflow Models** offer a convention for packaging machine learning models in multiple flavors, and a variety of tools to help you deploy them. Each Model is saved as a directory containing arbitrary files and a descriptor file that lists several “flavors” the model can be used in. For example, a TensorFlow model can be loaded as a TensorFlow DAG, or as a Python function to apply to input data. MLflow provides tools to deploy many common model types to diverse platforms: for example, any model supporting the “Python function” flavor can be deployed to a Docker-based REST server, to cloud platforms such as Azure ML and AWS SageMaker, and as a user-defined function in Apache Spark for batch and streaming inference. If you output MLflow Models using the Tracking API, MLflow will also automatically remember which Project and run they came from.


- **Mlflow Nodel Registry** A module that handles the management of models in MLflow and its life cycle, including state

In [3]:
import os
import mlflow
from mlflow import log_metric, log_param, log_artifact
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

### Exploring MLflow projects

An MLflow project represents the basic unit of organization of ML projects. There are three different environments supported by MLflow projects: 
- Conda environment
- Virtualenv environment
- Docker
- System environment

### Project Directories

When running an MLflow Project directory or repository that does not contain an MLproject file, MLflow uses the following conventions to determine the project’s attributes:

- The project’s name is the name of the directory.

- The Conda environment is specified in `conda.yaml`, if present. If no `conda.yaml` file is present, MLflow uses a Conda environment containing only Python (specifically, the latest Python available to Conda) when running the project.

- Any `.py` and `.sh` file in the project can be an entry point. MLflow uses Python to execute entry points with the `.py` extension, and it uses bash to execute entry points with the .sh extension. For more information about specifying project entrypoints at runtime, see Running Projects.

By default, entry points do not have any parameters when an MLproject file is not included. Parameters can be supplied at runtime via the mlflow run CLI or the `mlflow.projects.run()` Python API. Runtime parameters are passed to the entry point on the command line using `--key` value syntax. For more information about running projects and with runtime parameters, see Running Projects.

So, lest start making an example using Docker!!

**1.Create a folder for your project.** For this exercise we will call it `iris-classifier`.

In [ ]:
!mkdir iris-classifier

**2.Create the project's config file**

In [11]:
%%writefile ./iris-classifier/MLproject
name: Iris Classifier
docker_env:
  image: iris-classifier-docker
entry_points:
  main:
    command: "python train.py"

Writing ./iris-classifier/MLproject


**3. Add a Docker file to the project.**
Additionally, you can specify the Docker registry URL of your image. The advantage of running Docker is that your project is not bound to the Python language, as we will see in the advanced section of this book. The MLflow API is available in a Rest interface alongside the official clients: Python, Java, and R:

In [12]:
%%writefile ./iris-classifier/Dockerfile
FROM continuumio/miniconda:4.5.4
RUN pip install mlflow==1.11.0 \
    && pip install numpy==1.14.3 \
    && pip install scipy \
    && pip install pandas==0.22.0 \
    && pip install scikit-learn==0.20.4 \
    && pip install cloudpickle \
    && pip install pandas_datareader>=0.8.0

Writing ./iris-classifier/Dockerfile


 The preceding Docker image file is based on the open source package Miniconda, a free minimal installer with a minimal set of packages for data science 